# Analyze the alignment

In [ ]:
import collections

import Bio.SeqIO

import pandas as pd

In [ ]:
alignmentfile = "results/alignment.fa"

In [ ]:
alignment = {str(s.id): str(s.seq) for s in Bio.SeqIO.parse(alignmentfile, "fasta")}

In [ ]:
# none of the sites 3778, 8388, 8987, or 29871 are ones where Wuhan-Hu-1 differs from RaTG13
verified_mismatches = {  # in Wuhan-Hu-1 numbering
    **{
        site: "true gap relative to Wuhan-Hu-1 (before first site in submitted sequence numbering); this is in 5' UTR"
        for site in range(1, 16)
    },
    3778: "true a>g mismatch relative to Wuhan-Hu-1 (3763 in submitted sequence numbering); this is in ORF1a",
    8388: "true a>g mismatch relative to Wuhan-Hu-1 (8373 in submitted sequence numbering); this is in ORF1a",
    8987: "true t>a mismatch relative to Wuhan-Hu-1 (8972 in submitted sequence numbering); this is in ORF1a",
    **{
        site: "true gap relative to Wuhan-Hu-1 (site 1100 in submitted sequence numbering); this is in ORF1a"
        for site in range(11116, 11201)
    },
    **{
        site: "true gap relative to Wuhan-Hu-1 (site 27613 in submitted sequence numbering); this is in ORF7a-b"
        for site in range(27714, 27801)
    },
    29871: "true a>t mismatch relative to Wuhan-Hu-1 (last site, 29684, in submitted sequence numbering); this is in 3' UTR",
    **{
        site: "true gap relative to Wuhan-Hu-1 (after last site in submitted sequencing numbering); this is in 3' UTR"
        for site in range(29872, 29904)
    },
}

In [ ]:
site = {seqname: 0 for seqname in alignment}

alignment_length = len(list(alignment.values())[0])
assert all(alignment_length == len(s) for s in alignment.values())

d = collections.defaultdict(list)
for i in range(alignment_length):
    d["alignment_position"].append(i + 1)
    for seqname, seq in alignment.items():
        if seq[i] != "-":
            assert seq[i] in "acgt", seq[i]
            site[seqname] += 1
            d[f"{seqname} site"].append(site[seqname])
            d[f"{seqname} nt"].append(seq[i])
        else:
            d[f"{seqname} site"].append(site[seqname])
            d[f"{seqname} nt"].append("-")
df = pd.DataFrame(d)

assert set(verified_mismatches).issubset(
    df[df["SARS-CoV-2_28Dec2019_submission nt"] != df["NC_045512.2 nt"]]["NC_045512.2 site"]
)

assert not len(
    df[df["SARS-CoV-2_28Dec2019_submission nt"] != df["NC_045512.2 nt"]]
    .query("`NC_045512.2 site` not in @verified_mismatches")
)

In [ ]:
for other in list(alignment)[1: ]:
    print(f"\nDifferences with {other}")
    diffs = (
        df[df["SARS-CoV-2_28Dec2019_submission nt"] != df[f"{other} nt"]]
        .groupby(["SARS-CoV-2_28Dec2019_submission nt", "SARS-CoV-2_28Dec2019_submission site"], as_index=False, sort=False)
        .aggregate(
            reference_site=pd.NamedAgg(
                "NC_045512.2 site",
                lambda s: list(s)[0] if len(s) == 1 else f"{list(s)[0]}-{list(s)[-1]}",
            )
        )
        .merge(df.rename(columns={"NC_045512.2 site": "reference_site"}), how="left")
        .drop(columns=["SARS-CoV-2_28Dec2019_submission site", "alignment_position", "GWHABKF00000001 site"])
    )
    display(diffs)